In [89]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
from rff.layers import GaussianEncoding #pip install random-fourier-features-pytorch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler


import sys
import importlib
sys.path.append('../../helpers/')
import helper
# sys.path.append('../../model/')
# from modified_emb_model import Classifier, train, test

In [90]:
# Run regardless if you do or do not have GPU so all tensors are moved to right location later on
if torch.cuda.is_available():
    device_in_use = torch.device("cuda")
    print("GPU is available and being used")
else:
    device_in_use = torch.device("cpu")
    print("GPU is not available, using CPU instead")

GPU is not available, using CPU instead


In [91]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
df_val = pd.read_csv('./data/validation.csv')

In [92]:
print(len(df_train.columns))
df_train.columns

10


Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity', 'median_house_value'],
      dtype='object')

In [93]:
def categorize_columns(dataframe):
    categorical_columns = []
    continuous_columns = []
    unique_classes_per_column = []  # To hold the number of unique classes for each categorical column

    for column in dataframe.columns:
        if dataframe[column].dtype == 'object' or len(dataframe[column].unique()) <= 10:
            # If the column's data type is 'object' or it has 10 or fewer unique values, consider it categorical.
            categorical_columns.append(column)
            unique_classes_per_column.append(dataframe[column].nunique())  # Store the number of unique classes
        else:
            # Otherwise, consider it continuous.
            continuous_columns.append(column)

    # Calculate the total number of unique classes across all categorical columns.
    total_unique_classes = sum(dataframe[col].nunique() for col in categorical_columns)

    return categorical_columns, continuous_columns, total_unique_classes, unique_classes_per_column


cat_cols, cont_cols, total_unique, unique_classes_per_column = categorize_columns(df_train)
print(cat_cols)
print(cont_cols)
print(total_unique)
print(unique_classes_per_column)

['ocean_proximity']
['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']
5
[5]


In [94]:
df_train['ocean_proximity'].value_counts()

ocean_proximity
0.0    5423
1.0    3898
4.0    1558
3.0    1378
2.0       2
Name: count, dtype: int64

In [95]:
class SingleTaskDataset(Dataset):
    def __init__(self, df : pd.DataFrame, task1_column):
        self.n = df.shape[0]
        
        self.task1_labels = df[task1_column].astype(np.int64).values

        self.x = df.drop(task1_column, axis=1).astype(np.float32).values

   

    def __len__(self):
        return self.n
    
    def __getitem__(self, idx):
        # Retrieve features and labels from the dataframe using column names
        features = self.x[idx]
        labels_task1 = self.task1_labels[idx]

        return features, labels_task1
        # return self.x[index], self.task1_labels[index], self.task2_labels[index]

train_dataset = SingleTaskDataset(df_train, 'median_house_value')
val_dataset = SingleTaskDataset(df_val, 'median_house_value')
test_dataset = SingleTaskDataset(df_test, 'median_house_value')

batch_size = 32

# Wrapping with DataLoader for easy batch extraction
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [96]:
class CatNumTaskDataset(Dataset):
    def __init__(self, df: pd.DataFrame, task1_column, categorical_columns, numerical_columns):
        self.n = df.shape[0]
        self.task1_labels = df[task1_column].astype(np.int64).values
        
        # Extract categorical and numerical features
        self.x_categ = df[categorical_columns].astype(np.int64).values
        self.x_numer = df[numerical_columns].astype(np.float32).values

    def __len__(self):
        return self.n
    
    def __getitem__(self, idx):
        # Retrieve features and labels
        categ_features = self.x_categ[idx]
        numer_features = self.x_numer[idx]
        labels_task1 = self.task1_labels[idx]

        return categ_features, numer_features, labels_task1
    
# determine cont and cat columns
cat_cols, cont_cols, num_unique_cats, unique_classes_per_column = categorize_columns(df_train)
    
train_dataset = CatNumTaskDataset(df_train, 'median_house_value', cat_cols, cont_cols)
val_dataset = CatNumTaskDataset(df_val, 'median_house_value', cat_cols, cont_cols)
test_dataset = CatNumTaskDataset(df_test, 'median_house_value', cat_cols, cont_cols)

batch_size = 1064

# Wrapping with DataLoader for easy batch extraction
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

### Model init

In [97]:
num_features =  9
num_targets =   1
classes_per_target = [0] # 0 for single target regression task

print(f"Features: {num_features}")
print(f"Targets: {num_targets}")
print(f"Classes per Target: {classes_per_target}")

Features: 9
Targets: 1
Classes per Target: [0]


In [98]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(MultiHeadAttention, self).__init__()

        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads
        assert(self.head_dim * heads == embed_size), "Embed size needs to be div by heads"
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys =nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads*self.head_dim, embed_size)


    def forward(self, values, keys, query):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim=3) #(batch_size, head_dim, #query_embeddings, #key_embeddings)

        # Calculate simplified attention scores
        avg_attention = attention.mean(dim=0)  # Average across batches
        # print("batch average", avg_attention.shape)
        avg_attention = avg_attention.mean(dim=0).squeeze(dim=0)
        # print("head average", avg_attention.shape)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(N, query_len, self.heads*self.head_dim) #(batch_size, n_features, embed_size)
        out = self.fc_out(out)

        return out, avg_attention
    
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion, pre_norm_on):
        super(TransformerBlock, self).__init__()

        self.pre_norm_on = pre_norm_on
        if self.pre_norm_on:
            self.pre_norm = nn.LayerNorm(embed_size)
        self.attention = MultiHeadAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(nn.Linear(embed_size, forward_expansion*embed_size),
                                          nn.ReLU(),
                                          nn.Linear(forward_expansion*embed_size, embed_size)
                                          )
        self.dropout = nn.Dropout(dropout)

    def forward(self,value,key,query):
        if self.pre_norm_on:
            query = self.pre_norm(query)
            key = self.pre_norm(key)
            value = self.pre_norm(value)
            
        attention, avg_attention = self.attention(value, key, query)
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out, avg_attention
    
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, pre_norm_on):
        super(DecoderBlock, self).__init__()

        self.attention = MultiHeadAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(embed_size, heads, dropout, forward_expansion, pre_norm_on)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key):
        out, avg_attention = self.transformer_block(value, key, x)

        return out, avg_attention

class Decoder(nn.Module):
    def __init__(self,
                 embed_size,
                 num_layers,
                 heads,
                 forward_expansion,
                 decoder_dropout,
                 pre_norm_on
    ):
        super(Decoder, self).__init__()

        self.layers = nn.ModuleList(
                [
                    DecoderBlock(
                        embed_size,
                        heads,
                        dropout=decoder_dropout,
                        forward_expansion=forward_expansion,
                        pre_norm_on=pre_norm_on
                    )
                    for _ in range(num_layers)
                ]
            )
        self.avg_attention = None

    def forward(self, class_embed, context):
        for layer in self.layers:
            # x is the classification embedding (CLS Token)
            # context are the feature embeddings that will be used as key and value
            x, self.avg_attention = layer(class_embed, context, context)
  
        return x 

class Embeddings(nn.Module):
    def __init__(self, sigma, embed_size, input_size, embedding_dropout, n_cat_features, n_cont_features, num_target_labels, rff_on):
        super(Embeddings, self).__init__()

        self.rff_on = rff_on

        if self.rff_on:
            self.rffs = nn.ModuleList([GaussianEncoding(sigma=sigma, input_size=input_size, encoded_size=embed_size//2) for _ in range(n_cont_features)])
            self.dropout = nn.Dropout(embedding_dropout)
            self.mlp_in = embed_size
        else:
            self.mlp_in = input_size

        
        
        self.cont_embeddings = nn.ModuleList([nn.Linear(in_features=self.mlp_in, out_features=embed_size) for _ in range(n_cont_features)])

        self.n_cat_features = n_cat_features
        self.cat_embeddings = nn.ModuleList(nn.Embedding(num_embeddings=n_cat_features[_], embedding_dim=embed_size) for _ in range(len(n_cat_features)))

        # Classifcation Embeddings for each target label
        self.target_label_embeddings = nn.ModuleList([nn.Embedding(1, embed_size) for _ in range(num_target_labels)])


    def forward(self, x_cat, x_cont):
        # print(x_cat)
        # print(x_cat.unsqueeze(2))
        # print(x_cont)
        # print(x_cont.unsqueeze(2))
        x_cat = x_cat.unsqueeze(2) #(batch_size, n_features) -> (batch_size, n_features, 1)
        x_cont = x_cont.unsqueeze(2)

        rff_vectors = []
        if self.rff_on:
            for i, r in enumerate(self.rffs):
                input = x_cont[:,i,:]
                out = r(input)
                rff_vectors.append(out)
        
            x_cont = torch.stack(rff_vectors, dim=1)
        
        embeddings = []
        for i, e in enumerate(self.cont_embeddings):
            # print(x_cont.shape)
            goin_in = x_cont[:,i,:]
            goin_out = e(goin_in)
            # print(f'cont going out : {goin_out.shape}')
            embeddings.append(goin_out)

        # for i, e in enumerate(self.cat_embeddings):
        #     # print(x_cat.shape)
        #     goin_in = x_cat[:,i,:]
        #     # print(f' in: {goin_in.shape}')

        #     # the nn.Embeding module returns shape [bs, i, emb_dim] to allow accessing these embeddigns by an index
        #     # I am assuming that we don't need to indecies, just the embeddings so I flatten

        #     goin_out = e(goin_in) 
        #     # print(f'cat going out : {goin_out}')
        #     embeddings.append(goin_out.squeeze(1))
        for i, e in enumerate(self.cat_embeddings):
            goin_in = x_cat[:, i, :]
            goin_out = e(goin_in)
            # print(goin_out.shape)
            embeddings.append(goin_out.squeeze(1))

        target_label_embeddings_ = []
        for e in self.target_label_embeddings:
            input = torch.tensor([0], device=x_cat.device)
            temp = e(input)
            temp = temp.repeat(x_cat.size(0), 1)
            temp = temp.unsqueeze(1)
            target_label_embeddings_.append(temp)

        class_embeddings = torch.stack(target_label_embeddings_, dim=1)
        
        # class_embed = self.classification_embedding(torch.tensor([0], device=x.device))  # use index 0 for the classification embedding
        # class_embed = class_embed.repeat(x.size(0), 1) # -> (batch_size, embed_size)
        # class_embed = class_embed.unsqueeze(1)

        context = torch.stack(embeddings, dim=1)

        return class_embeddings, context

class classificationHead(nn.Module):
    def __init__(self, embed_size, dropout, mlp_scale_classification, num_target_classes):
        super(classificationHead, self).__init__()
        
        #flattening the embeddings out so each sample in batch is represented with a 460 dimensional vector
        self.input = embed_size
        self.lin1 = nn.Linear(self.input, mlp_scale_classification*self.input)
        self.drop = nn.Dropout(dropout)
        # self.lin2 = nn.Linear(2*self.input, 2*self.input)
        self.lin3 = nn.Linear(mlp_scale_classification*self.input, self.input)
        self.lin4 = nn.Linear(self.input, num_target_classes)
        self.relu = nn.ReLU()
        self.initialize_weights()

    def initialize_weights(self): #he_initialization.
        torch.nn.init.kaiming_normal_(self.lin1.weight, nonlinearity='relu')
        torch.nn.init.zeros_(self.lin1.bias)

        torch.nn.init.kaiming_normal_(self.lin3.weight, nonlinearity='relu')
        torch.nn.init.zeros_(self.lin3.bias)

    def forward(self, x):

        x= torch.reshape(x, (-1, self.input))

        x = self.lin1(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.lin3(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.lin4(x)
  
        return x
    
class regressionHead(nn.Module):
    def __init__(self, embed_size, dropout, mlp_scale_regression):
        super(regressionHead, self).__init__()
        
        self.input = embed_size
        self.lin1 = nn.Linear(self.input, mlp_scale_regression * self.input)
        self.drop = nn.Dropout(dropout)
        self.lin2 = nn.Linear(mlp_scale_regression * self.input, self.input)
        self.lin3 = nn.Linear(self.input, 1)  # Output layer for regression (single output)
        self.relu = nn.ReLU()
        self.initialize_weights()

    def initialize_weights(self): # the initialization
        torch.nn.init.kaiming_normal_(self.lin1.weight, nonlinearity='relu')
        torch.nn.init.zeros_(self.lin1.bias)

        torch.nn.init.kaiming_normal_(self.lin2.weight, nonlinearity='relu')
        torch.nn.init.zeros_(self.lin2.bias)

    def forward(self, x):
        x = torch.reshape(x, (-1, self.input))

        x = self.lin1(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.lin2(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.lin3(x) 
        return x    


# DEFAULT PARAMETERS SET UP FOR MY DATASET. BE CAREFUL AND MAKE SURE YOU SET THEM UP HOW YOU WANT.
# All dropout is initially turned off
class Classifier(nn.Module):
    def __init__(self, 
                 rff_on = False,
                 sigma=4,
                 embed_size=20,
                 input_size=1,
                 embedding_dropout = 0,
                 n_cat_features=66, # YOU WILL PROBABLY NEED TO CHANGE
                 n_cont_features=2, # YOU WILL PROBABLY NEED TO CHANGE
                 num_layers=1,
                 heads=1,
                 forward_expansion=4, # Determines how wide the MLP is in the encoder. Its a scaling factor. 
                 decoder_dropout=0,
                 classification_dropout = 0,
                 pre_norm_on = False,
                 mlp_scale_classification = 4, #widens the mlp in the classification heads
                 targets_classes : list= [0] #[12,2] #put the number of classes in each target variable. traffic type = 3 classes, application type = 8 classes
                 ):
        super(Classifier, self).__init__()

        self.embeddings = Embeddings(rff_on=rff_on, sigma=sigma, embed_size=embed_size, input_size=input_size, embedding_dropout=embedding_dropout, n_cat_features=n_cat_features, n_cont_features=n_cont_features, num_target_labels=len(targets_classes))
        self.decoder = Decoder(embed_size=embed_size, num_layers=num_layers, heads=heads, forward_expansion=forward_expansion, decoder_dropout=decoder_dropout, pre_norm_on=pre_norm_on)
        self.classifying_heads = nn.ModuleList([regressionHead(embed_size=embed_size, dropout=classification_dropout, mlp_scale_regression=mlp_scale_classification)])
        
    def forward(self, x_cat, x_num):
        
        class_embed, context = self.embeddings(x_cat, x_num)
        


        
        
        
        ########################
        
        # class_embed, context = self.embeddings(x)

        x = self.decoder(class_embed, context)
        
        probability_dist_raw = []
        for i, e in enumerate(self.classifying_heads):
            input = x[:, i,:]
            output = e(input)
            probability_dist_raw.append(output)
        
        return probability_dist_raw


# Training and Testing Loops
# Should not need modification
def train(dataloader, model, loss_functions : list, optimizer, device_in_use):
  total_loss = 0
  target_count = len(loss_functions)

  total_correct = [0] * target_count
  total_samples = [0] * target_count
  all_targets   = [[] for _ in range(target_count)]
  all_predictions = [[] for _ in range(target_count)]
  y_pred_softmax = [0] * target_count
  y_pred_labels = [0] * target_count
  accuracy = []

  model.train()

  for (x_num, x_cat, targets) in dataloader:
      
    x_cat, x_num, targets = x_cat.to(device_in_use), x_num.to(device_in_use), targets.float().to(device_in_use)
    task_predictions = model(x_num, x_cat) #contains a list of the tensor outputs for each task

    print(task_predictions)
    loss = 0
    # print(len(loss_functions))
    for i in range(len(loss_functions)):
    #   print('yp')
    #   print(i)
    #   print(loss_functions[0])
    #   print(task_predictions[0].dtype)
    #   print(targets.dtype)
      loss += loss_functions[0](task_predictions[0], targets)
    loss = loss/len(loss_functions)
    total_loss+= loss.item() #just summing the two losses
    
    mse_1 = ((task_predictions[0] - targets)**2).mean()

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  #Loss
  avg_loss = total_loss/len(dataloader)
  #Accuracies
  


  return avg_loss, mse_1





def test(dataloader, model, loss_functions : list, device_in_use):
  total_loss = 0
  target_count = len(loss_functions)
  total_correct = [0] * target_count
  total_samples = [0] * target_count
  all_targets   = [[] for _ in range(target_count)]
  all_predictions = [[] for _ in range(target_count)]
  y_pred_softmax = [0] * target_count
  y_pred_labels = [0] * target_count
  accuracy = []
  f1       = []

  model.eval()

  with torch.no_grad():
    for (x_num, x_cat, targets) in dataloader:
      x_cat, x_num, targets = x_cat.to(device_in_use), x_num.to(device_in_use), targets.float().to(device_in_use)

      task_predictions = model(x_num, x_cat) #contains a list of the tensor outputs for each task
      

      loss = 0
      for i in range(len(loss_functions)):
        loss += loss_functions[0](task_predictions[0], targets)
    #   loss = loss/len(loss_functions)
    #   print(loss)
      total_loss+= loss.item()

      mse_1 = ((task_predictions[0] - targets)**2).mean()

  #Loss
  avgloss = total_loss/len(dataloader)
 
  #Recall
  # recall = recall_score(all_targets, all_predictions, average='weighted')


  return avgloss, mse_1


In [99]:
print(len(cat_cols), len(cont_cols))

1 9


In [113]:
#Updated with f1
model = Classifier(n_cat_features=unique_classes_per_column,
                   n_cont_features=len(cont_cols)-1,
                                   pre_norm_on=True, 
                                   rff_on=True, 
                                   forward_expansion=1, 
                                   mlp_scale_classification=2, 
                                   targets_classes=classes_per_target
                                   ).to(device_in_use) # Instantiate the model

In [114]:
loss_functions = []
loss_functions.append(nn.MSELoss())

optimizer = torch.optim.Adam(params=model.parameters(), lr = .001) # Maybe try messing around with optimizers. try other torch optimizers with different configurations.

In [115]:
def format_metric(value): # Used to format the metrics output
    return f"{value:.5f}"

In [116]:
epochs = 20

train_losses = []
test_losses = []
train_accuracies = []
test_accuracies =  []

for t in range(epochs):
  train_loss, train_mse= train(train_dataloader, model, loss_functions, optimizer, device_in_use=device_in_use)
  test_loss, test_mse = test(test_dataloader, model, loss_functions, device_in_use=device_in_use)

  # Training metrics 
  train_losses.append(train_loss)
  train_accuracies.append(train_mse.item())

  # Test metrics 
  test_losses.append(test_loss)
  test_accuracies.append(test_mse)

  epoch_str = f"Epoch [{t+1:2}/{epochs}]"
  # print(train_accuracies[0])
  train_metrics = f"Train: Loss {format_metric(train_loss)},  MSE {format_metric(train_mse)}"
  test_metrics = f"Test: Loss {format_metric(test_loss)},  MSE  {format_metric(test_mse)}"
  print(f"{epoch_str:20} | {train_metrics:65} | {test_metrics}")



# Plotting the loss curves
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(range(1, epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, epochs+1), [l for l in test_losses], label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss Curve')
plt.legend()

# Plotting the accuracy curves
plt.subplot(1, 3, 2)
# print(train_accuracies_1.dtype)
plt.plot(range(1, epochs+1), train_accuracies, label='Train MSE')
plt.plot(range(1, epochs+1), test_accuracies, label='Test MSE')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Test MSE Curve for Flow Bytes')
plt.legend()


Epoch [ 1/20]        | Train: Loss 0.50631,  MSE 0.51875                                 | Test: Loss 0.49928,  MSE  0.46826
Epoch [ 2/20]        | Train: Loss 0.50464,  MSE 0.47226                                 | Test: Loss 0.49001,  MSE  0.52902
Epoch [ 3/20]        | Train: Loss 0.49801,  MSE 0.49767                                 | Test: Loss 0.48708,  MSE  0.44522
Epoch [ 4/20]        | Train: Loss 0.49340,  MSE 0.43619                                 | Test: Loss 0.48792,  MSE  0.52895
Epoch [ 5/20]        | Train: Loss 0.49370,  MSE 0.47367                                 | Test: Loss 0.48633,  MSE  0.47843
Epoch [ 6/20]        | Train: Loss 0.49546,  MSE 0.48808                                 | Test: Loss 0.48495,  MSE  0.48746
Epoch [ 7/20]        | Train: Loss 0.49546,  MSE 0.51403                                 | Test: Loss 0.48427,  MSE  0.47678
Epoch [ 8/20]        | Train: Loss 0.49563,  MSE 0.53047                                 | Test: Loss 0.48535,  MSE  0.50502
